In [111]:
import pandas as pd
import numpy as np
from tqdm import tqdm
pd.set_option('mode.chained_assignment', None)

In [112]:
from os import listdir
from os.path import isfile, join
from datetime import datetime
from dateutil.parser import parse

In [113]:
confirmed_global_file = 'C:/Users/terra/corona_data/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
confirmed_states_file = 'C:/Users/terra/corona_data/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'

recovered_file = 'C:/Users/terra/corona_data/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'

deaths_global_file = 'C:/Users/terra/corona_data/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
deaths_states_file = 'C:/Users/terra/corona_data/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'

In [114]:
confirmed_global_df = pd.read_csv(confirmed_global_file)
confirmed_states_df = pd.read_csv(confirmed_states_file)
recovered_df = pd.read_csv(recovered_file)
deaths_states_df = pd.read_csv(deaths_states_file)
deaths_global_df = pd.read_csv(deaths_global_file)
confirmed_global_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,110,110,120,170,174,237,273,281,299,349
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,186,197,212,223,243,259,277,304,333,361
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,409,454,511,584,716,847,986,1171,1251,1320
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,267,308,334,370,376,390,428,439,466,501
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,4,5,7,7,7,8,8,8,10,14


#### Remove US from Global Data

In [115]:
confirmed_global_df = confirmed_global_df[confirmed_global_df['Country/Region'] != 'US']
deaths_global_df = deaths_global_df[deaths_global_df['Country/Region'] != "US"]

In [116]:
confirmed_global_df[confirmed_global_df['Country/Region'] == 'US']

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20


#### Get list of dates

In [117]:
dates = confirmed_global_df.columns[4:]
dates

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20',
       '2/3/20', '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20',
       '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20',
       '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20',
       '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20',
       '2/28/20', '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20', '3/5/20',
       '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20', '3/11/20', '3/12/20',
       '3/13/20', '3/14/20', '3/15/20', '3/16/20', '3/17/20', '3/18/20',
       '3/19/20', '3/20/20', '3/21/20', '3/22/20', '3/23/20', '3/24/20',
       '3/25/20', '3/26/20', '3/27/20', '3/28/20', '3/29/20', '3/30/20',
       '3/31/20', '4/1/20', '4/2/20', '4/3/20', '4/4/20', '4/5/20'],
      dtype='object')

#### Replace NaN Values with Country Region if Province/State or Admin2 is empty

In [118]:
### Remove NaN values from Province/State Column
confirmed_global_df.loc[confirmed_global_df['Province/State'].isnull(),'Province/State'] = confirmed_global_df['Country/Region']
confirmed_states_df.loc[confirmed_states_df['Province_State'].isnull(),'Province_State'] = confirmed_states_df['Country_Region']
confirmed_states_df.loc[confirmed_states_df['Admin2'].isnull(),'Admin2'] = confirmed_states_df['Province_State']

deaths_global_df.loc[deaths_global_df['Province/State'].isnull(),'Province/State'] = deaths_global_df['Country/Region']
deaths_states_df.loc[deaths_states_df['Province_State'].isnull(),'Province_State'] = deaths_states_df['Country_Region']
deaths_states_df.loc[deaths_states_df['Admin2'].isnull(),'Admin2'] = deaths_states_df['Province_State']

recovered_df.loc[recovered_df['Province/State'].isnull(),'Province/State'] = recovered_df['Country/Region']

#### Remove '.' at end of locations/regions/states

In [119]:

confirmed_global_df['Province/State'] = confirmed_global_df['Province/State'].str.replace('.','')
confirmed_states_df['Province_State'] = confirmed_states_df['Province_State'].str.replace('.','')
confirmed_states_df['Admin2'] = confirmed_states_df['Admin2'].str.replace('.','')

deaths_global_df['Province/State'] = deaths_global_df['Province/State'].str.replace('.','')
deaths_states_df['Province_State'] = deaths_states_df['Province_State'].str.replace('.','')
deaths_states_df['Admin2'] = deaths_states_df['Admin2'].str.replace('.','')

recovered_df['Province/State'] = recovered_df['Province/State'].str.replace('.','')


### Post Object

Holds a list of documents that will be posted to the database

In [120]:
post = {'data': []}

### DATA

In [121]:


for x in tqdm(dates):
    print(f'Creating object for: {x}' )
    date_obj = {
        'date': pd.to_datetime(x),
        'total_confirmed': int(confirmed_global_df[x].sum() + confirmed_states_df[x].sum()),
        'total_deaths': int(deaths_global_df[x].sum() + deaths_states_df[x].sum()),
        'total_recovered': int(recovered_df[x].sum()),
        'locations': {}
    }
    print('Creating Global Confirmed Data...')
    for index, row in confirmed_global_df.iterrows():
        date = x
        location = row['Province/State']
        region = row['Country/Region']
        lat = row['Lat']
        lng = row['Long']
        confirmed = row[x]
        
        obj = {
            'region': region,
            'lat': lat,
            'lng': lng,
            'confirmed': confirmed
        }
        date_obj['locations'][location] = obj
    print('Creating State Confirmed Data...')
    for index, row in confirmed_states_df.iterrows():
        date = x
        location = row['Admin2']
        region = row['Country_Region']
        state = row['Province_State']
        lat = row['Lat']
        lng = row['Long_']
        confirmed = row[x]
        
        obj = {
            'region': region,
            'state': state,
            'lat': lat,
            'lng': lng,
            'confirmed': confirmed
        }
        date_obj['locations'][location] = obj
    post['data'].append(date_obj)
    print('Adding Global Death Data...')
    for index, row in deaths_global_df.iterrows():
        date = x
        location = row['Province/State']
        region = row['Country/Region']
        deaths = row[x]
 
        for i in post['data']:

            if i['date'] == pd.to_datetime(date):
                for y in i['locations']:
                    try:
                        if y[location] and y[location]['region'] == region:
                            y[location]['deaths'] = deaths
                    except:
                        pass
    print('Adding States Death Data...')
    for i, r in deaths_states_df.iterrows():
        date = x
        location = r['Admin2']
        state = r['Province_State']
        region = r['Country_Region']
        deaths = r[x]

        for i in post['data']:

            if i['date'] == pd.to_datetime(date):
                for y in i['locations']:
                    try: 
                        if y[location] and y[location]['state'] == state:
                            y[location]['deaths'] = deaths
                    except:
                        pass
        
    print('Adding Recovered Data...')
    for i, r in recovered_df.iterrows():
        date = x
        location = r['Province/State']
        region = r['Country/Region']
        recovered = r[x]

        for i in post['data']:

            if i['date'] == pd.to_datetime(date):
                for y in i['locations']:
                    try: 
                        if y[location] and y[location]['region'] == region:
                            y[location]['recovered'] = recovered
                    except:
                        pass
                    
print('Adding 0 values to locations with no recovered/death data')
for x in tqdm(post['data']):
    for key in x['locations']:
        try:
            if x['locations'][key]['recovered']:
                pass

        except:
            x['locations'][key]['recovered'] = 0

        try:
            if x['locations'][key]['deaths']:
                pass

        except:
            x['locations'][key]['deaths'] = 0

                
print('Deleting locations with no geographical area...')
for x in tqdm(post['data']):
    deletions = []
    for key in x['locations']:
        if x['locations'][key]['lat'] == 0.0 and x['locations'][key]['lng'] == 0.0:
            deletions.append(key)

    for y in deletions:
        if x['locations'][y]:
            del x['locations'][y]
            print(f'Deleted {y}')


  0%|                                                                                           | 0/75 [00:00<?, ?it/s]

Creating object for: 1/22/20
Creating Global Confirmed Data...
Creating State Confirmed Data...
Adding Global Death Data...
Adding States Death Data...
Adding Recovered Data...


  1%|█                                                                                  | 1/75 [00:04<05:06,  4.14s/it]

Creating object for: 1/23/20
Creating Global Confirmed Data...
Creating State Confirmed Data...
Adding Global Death Data...
Adding States Death Data...
Adding Recovered Data...


  3%|██▏                                                                                | 2/75 [00:08<05:16,  4.33s/it]

...

 96%|██████████████████████████████████████████████████████████████████████████████▋   | 72/75 [00:04<00:00, 14.35it/s]

Deleted Grand Princess
Deleted Diamond Princess
Deleted Recovered
Deleted MS Zaandam
Deleted Out of AL
Deleted Out of AK
Deleted Out of AZ
Deleted Out of AR
Deleted Out of CA
Deleted Out of CO
Deleted Out of CT
Deleted Out of DE
Deleted Out of DC
Deleted Out of FL
Deleted Out of GA
Deleted Out of HI
Deleted Out of ID
Deleted Out of IL
Deleted Out of IN
Deleted Out of IA
Deleted Out of KS
Deleted Out of KY
Deleted Out of LA
Deleted Out of ME
Deleted Out of MD
Deleted Out of MA
Deleted Out of MI
Deleted Out of MN
Deleted Out of MS
Deleted Out of MO
Deleted Out of MT
Deleted Out of NE
Deleted Out of NV
Deleted Out of NH
Deleted Out of NJ
Deleted Out of NM
Deleted Out of NY
Deleted Out of NC
Deleted Out of ND
Deleted Out of OH
Deleted Out of OK
Deleted Out of OR
Deleted Out of PA
Deleted Out of RI
Deleted Out of SC
Deleted Out of SD
Deleted Out of TN
Deleted Out of TX
Deleted Out of UT
Deleted Out of VT
Deleted Out of VA
Deleted Out of WA
Deleted Out of WV
Deleted Out of WI
Deleted Out of 

 99%|████████████████████████████████████████████████████████████████████████████████▉ | 74/75 [00:05<00:00, 14.63it/s]

Deleted Grand Princess
Deleted Diamond Princess
Deleted Recovered
Deleted MS Zaandam
Deleted Out of AL
Deleted Out of AK
Deleted Out of AZ
Deleted Out of AR
Deleted Out of CA
Deleted Out of CO
Deleted Out of CT
Deleted Out of DE
Deleted Out of DC
Deleted Out of FL
Deleted Out of GA
Deleted Out of HI
Deleted Out of ID
Deleted Out of IL
Deleted Out of IN
Deleted Out of IA
Deleted Out of KS
Deleted Out of KY
Deleted Out of LA
Deleted Out of ME
Deleted Out of MD
Deleted Out of MA
Deleted Out of MI
Deleted Out of MN
Deleted Out of MS
Deleted Out of MO
Deleted Out of MT
Deleted Out of NE
Deleted Out of NV
Deleted Out of NH
Deleted Out of NJ
Deleted Out of NM
Deleted Out of NY
Deleted Out of NC
Deleted Out of ND
Deleted Out of OH
Deleted Out of OK
Deleted Out of OR
Deleted Out of PA
Deleted Out of RI
Deleted Out of SC
Deleted Out of SD
Deleted Out of TN
Deleted Out of TX
Deleted Out of UT
Deleted Out of VT
Deleted Out of VA
Deleted Out of WA
Deleted Out of WV
Deleted Out of WI
Deleted Out of 

100%|██████████████████████████████████████████████████████████████████████████████████| 75/75 [00:05<00:00, 14.42it/s]


{'region': 'US',
 'state': 'South Dakota',
 'lat': 44.90608745,
 'lng': -103.5079318,
 'confirmed': 0,
 'recovered': 0,
 'deaths': 0}

In [122]:
import pymongo
import dns
username = 'Terra925'
password = 'H%40mmond271'

conn = 'mongodb+srv://' + username +':' + password + '@cluster0-paegd.mongodb.net/test?retryWrites=true&w=majority'
client = pymongo.MongoClient(conn)
db = client['corona_virus']
collection = db['cases_by_date']

In [123]:
for x in post['data']:
    id = collection.insert_one(x).inserted_id

In [81]:
post['data'][0]

{'date': Timestamp('2020-01-22 00:00:00'),
 'total_confirmed': 555,
 'total_deaths': 17,
 'total_recovered': 28,
 'locations': {'Afghanistan': {'region': 'Afghanistan',
   'lat': 33.0,
   'lng': 65.0,
   'confirmed': 0,
   'recovered': 0,
   'deaths': 0},
  'Albania': {'region': 'Albania',
   'lat': 41.1533,
   'lng': 20.1683,
   'confirmed': 0,
   'recovered': 0,
   'deaths': 0},
  'Algeria': {'region': 'Algeria',
   'lat': 28.0339,
   'lng': 1.6596,
   'confirmed': 0,
   'recovered': 0,
   'deaths': 0},
  'Andorra': {'region': 'Andorra',
   'lat': 42.5063,
   'lng': 1.5218,
   'confirmed': 0,
   'recovered': 0,
   'deaths': 0},
  'Angola': {'region': 'Angola',
   'lat': -11.2027,
   'lng': 17.8739,
   'confirmed': 0,
   'recovered': 0,
   'deaths': 0},
  'Antigua and Barbuda': {'region': 'Antigua and Barbuda',
   'lat': 17.0608,
   'lng': -61.7964,
   'confirmed': 0,
   'recovered': 0,
   'deaths': 0},
  'Argentina': {'region': 'Argentina',
   'lat': -38.4161,
   'lng': -63.6167,
   